<a href="https://colab.research.google.com/github/Anacatlisboa23/Tese/blob/main/MIC_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIC Prediction

In [30]:
# Importação das bibliotecas necessárias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import re
import csv

In [58]:
data= pd.read_csv("/content/eskape_bacteria_MIC.csv")

In [59]:
data.drop(columns="Unnamed: 0", inplace=True)

In [60]:
data

,Sequence,Name,Target_Organism,MIC,Function
0,VTSWSLCTPGCTSPGGGSNCSFCC,Microbisporicin A1 (Bacteriocin),L Staphylococcus aureus,MIC≤0.13 μg/ml,antibacterian group ESKAPE
1,VTSWSLCTPGCTSPGGGSNCSFCC,Microbisporicin A1 (Bacteriocin),L Enterococcus faecium,MIC=2 μg/ml,antibacterian group ESKAPE
2,MSWLNFLKYIAKYGKKAVSAAWKYKGKVLEWLNVGPTLEWVWQKLK...,Aureocin A53 (Bacteriocin),Staphylococcus aureus,NaN,antibacterian group ESKAPE
3,ATRSYGNGVYCNNSKCWVNWGEAKENIAGIVISGWASGLAGMGH,Enterocin P (Pediocin-like peptide; Bacteriocin),Enterococcus faecium,MIC=2 ng/ml,antibacterian group ESKAPE
4,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Staphylococcus aureus,MIC=0.2-0.8 µg/ml,antibacterian group ESKAPE
...,...,...,...,...,...
3619,KLLNLLPGLLAGIF,Reverse Pxt‐5,Staphylococcus aureus,NaN,antibacterian group ESKAPE
3620,ALLKLAPRLLAGIF,Reverse Pxt-2,Pseudomonas aeruginosa,NaN,antibacterian group ESKAPE
3621,ALLKLAPRLLAGIF,Reverse Pxt-2,Staphylococcus aureus,NaN,antibacterian group ESKAPE
3622,LLNSGVKLGTKLLSGLLN,Reverse Pxt-12,Pseudomonas aeruginosa,NaN,antibacterian group ESKAPE


In [52]:
data.dropna(subset="MIC", inplace=True)

2002 dados de MICs

# Tratar o dataset:

In [62]:
import pandas as pd
import re

def converter_valor(valor, unidade):
    conversoes = {
        'μg/ml': 1,
        'µg/ml': 1,
        'ng/ml': 1e-3,
        'nm': 1e-6,
        'µm': 1e-6,
        'μm': 1e-6,
        'mm': 1e-3,
        'g/l': 1e3,
        'mg/ml': 1e3,
        'µg/ml': 1,
        'μg/ml': 1,
        'mm': 1e-3,
        'g/l': 1e3
    }
    fator_conversao = conversoes.get(unidade.lower(), None)
    if fator_conversao is None:
        raise ValueError(f"Unidade desconhecida: {unidade}")
    return valor * fator_conversao




In [78]:
def converter_para_ug_ml(valor_str):
    try:
        if '-' in valor_str:
            partes = valor_str.split('-')
            valor_min = converter_para_ug_ml(partes[0].strip())
            valor_max = converter_para_ug_ml(partes[1].strip())
            if valor_min is not None and valor_max is not None:
                return (valor_min + valor_max) / 2  # Retornar a média dos valores
            return None

        operador = re.findall(r'[<>]=?|=', valor_str)
        operador = operador[0] if operador else ""
        valor_str = valor_str.replace(operador, '').strip()

        match = re.match(r'([0-9.]+)\s*([a-zA-Z/μµ]+)', valor_str)
        if not match:
            raise ValueError(f"Formato inválido: {valor_str}")

        valor = float(match.group(1))
        unidade = match.group(2)

        valor_convertido = converter_valor(valor, unidade)
        return valor_convertido

    except ValueError as e:
        print(f"Ignorando valor com erro: {e}")
        return None


In [79]:

def converter_row(row):
    if isinstance(row, str):
        return converter_para_ug_ml(row.replace('MIC', '').strip())
    return None

In [70]:

def processar_dataset(df):
    # Aplicar conversão para a coluna MIC
    df['MIC_padronizado'] = df['MIC'].apply(converter_row)

In [80]:
# Comparar valores numéricos e padronizados
def verificar(row):
    mic_numerico = row.get('MIC_numerico', None)
    mic_padronizado = row.get('MIC_padronizado', None)

    if mic_numerico is not None and mic_padronizado is not None:
        return abs(mic_numerico - mic_padronizado) < 1e-6
    return None

In [81]:
def processar_dataset(df):
    # Verificar a existência da coluna 'MIC'
    if 'MIC' not in df.columns:
        raise ValueError("Coluna 'MIC' não encontrada no DataFrame")

    # Aplicar conversão para a coluna MIC
    df['MIC_padronizado'] = df['MIC'].apply(converter_row)

    # Garantir que a coluna 'MIC_numerico' exista e não tenha valores nulos
    if 'MIC_numerico' not in df.columns:
        df['MIC_numerico'] = None

    # Comparar valores numéricos e padronizados
    df['Verificacao'] = df.apply(verificar, axis=1)

    return df



In [82]:
df = pd.DataFrame(data)
df = processar_dataset(df)

Ignorando valor com erro: Formato inválido: ≤0.13 μg/ml
Ignorando valor com erro: Formato inválido: 0.2
Ignorando valor com erro: Formato inválido: 6.1
Ignorando valor com erro: Formato inválido: 48.8
Ignorando valor com erro: Formato inválido: 3
Ignorando valor com erro: Formato inválido: 6
Ignorando valor com erro: Formato inválido: 125
Ignorando valor com erro: Formato inválido: 250
Ignorando valor com erro: Formato inválido: 250
Ignorando valor com erro: Formato inválido: 250
Ignorando valor com erro: Formato inválido: 250
Ignorando valor com erro: Formato inválido: 250
Ignorando valor com erro: Formato inválido: 125
Ignorando valor com erro: Formato inválido: 1.25
Ignorando valor com erro: Formato inválido: 2.5
Ignorando valor com erro: Formato inválido: 1.25
Ignorando valor com erro: Formato inválido: 5
Ignorando valor com erro: Formato inválido: 2.5
Ignorando valor com erro: Formato inválido: 17.7
Ignorando valor com erro: Formato inválido: 4
Ignorando valor com erro: Formato in

In [88]:
df.dropna(subset="MIC_padronizado", inplace= True)

In [89]:
df

,Sequence,Name,Target_Organism,MIC,Function,MIC_padronizado,MIC_numerico,Verificacao
1,VTSWSLCTPGCTSPGGGSNCSFCC,Microbisporicin A1 (Bacteriocin),L Enterococcus faecium,MIC=2 μg/ml,antibacterian group ESKAPE,2.000000e+00,None,None
3,ATRSYGNGVYCNNSKCWVNWGEAKENIAGIVISGWASGLAGMGH,Enterocin P (Pediocin-like peptide; Bacteriocin),Enterococcus faecium,MIC=2 ng/ml,antibacterian group ESKAPE,2.000000e-03,None,None
6,SNDSLWYGVGQFMGKQANCITNHPVKHMIIPGYCLSKILG,Enterocin X alpha (Two-peptide bacteriocin),Enterococcus faecium 1344L,MIC=50.8 nM,antibacterian group ESKAPE,5.080000e-05,None,None
10,NRWYCNSAAGGVGGAAVCGLAGYVGEAKENIAGEVRKGWGMAGGFT...,Enterocin E-760 (Bacteriocin),Klebsiella pneumoniae,MIC=3.2 μg/ml,antibacterian group ESKAPE,3.200000e+00,None,None
11,LIDHLGAPRWAVDTILGAIAVGNLASWVLALVPGPGWAVKAGLATA...,Lactocyclicin Q (Bacteriocin),Enterococcus faecium,MIC=0.71 µM,antibacterian group ESKAPE,7.100000e-07,None,None
...,...,...,...,...,...,...,...,...
3603,GLLKKLLKIAAKVGKKLL,"XT-7 [G4K,P5K,S15K,N16K]",Staphylococcus aureus,MIC=6 µM,antibacterian group ESKAPE,6.000000e-06,None,None
3607,GLKEVAHSAKKFAKGFISGLTGS,Pxt‐3,Pseudomonas aeruginosa,MIC >256 µM,antibacterian group ESKAPE,2.560000e-04,None,None
3608,GLKEVAHSAKKFAKGFISGLTGS,Pxt‐3,Staphylococcus aureus,MIC >256 µM,antibacterian group ESKAPE,2.560000e-04,None,None
3616,NLLGSLLKTGLKVGSNLL,Pxt‐12(CPF-St7),Pseudomonas aeruginosa,MIC >256 µM,antibacterian group ESKAPE,2.560000e-04,None,None


1905 dados

In [90]:
df.drop(columns=["MIC_numerico","Verificacao"])

,Sequence,Name,Target_Organism,MIC,Function,MIC_padronizado
1,VTSWSLCTPGCTSPGGGSNCSFCC,Microbisporicin A1 (Bacteriocin),L Enterococcus faecium,MIC=2 μg/ml,antibacterian group ESKAPE,2.000000e+00
3,ATRSYGNGVYCNNSKCWVNWGEAKENIAGIVISGWASGLAGMGH,Enterocin P (Pediocin-like peptide; Bacteriocin),Enterococcus faecium,MIC=2 ng/ml,antibacterian group ESKAPE,2.000000e-03
6,SNDSLWYGVGQFMGKQANCITNHPVKHMIIPGYCLSKILG,Enterocin X alpha (Two-peptide bacteriocin),Enterococcus faecium 1344L,MIC=50.8 nM,antibacterian group ESKAPE,5.080000e-05
10,NRWYCNSAAGGVGGAAVCGLAGYVGEAKENIAGEVRKGWGMAGGFT...,Enterocin E-760 (Bacteriocin),Klebsiella pneumoniae,MIC=3.2 μg/ml,antibacterian group ESKAPE,3.200000e+00
11,LIDHLGAPRWAVDTILGAIAVGNLASWVLALVPGPGWAVKAGLATA...,Lactocyclicin Q (Bacteriocin),Enterococcus faecium,MIC=0.71 µM,antibacterian group ESKAPE,7.100000e-07
...,...,...,...,...,...,...
3603,GLLKKLLKIAAKVGKKLL,"XT-7 [G4K,P5K,S15K,N16K]",Staphylococcus aureus,MIC=6 µM,antibacterian group ESKAPE,6.000000e-06
3607,GLKEVAHSAKKFAKGFISGLTGS,Pxt‐3,Pseudomonas aeruginosa,MIC >256 µM,antibacterian group ESKAPE,2.560000e-04
3608,GLKEVAHSAKKFAKGFISGLTGS,Pxt‐3,Staphylococcus aureus,MIC >256 µM,antibacterian group ESKAPE,2.560000e-04
3616,NLLGSLLKTGLKVGSNLL,Pxt‐12(CPF-St7),Pseudomonas aeruginosa,MIC >256 µM,antibacterian group ESKAPE,2.560000e-04


Guardar dataset:

In [91]:
df.to_csv("/content/MIC_ESKAPE.csv", index=False)

# Machinne Learning:

In [ ]:
# Separando as features (X) e o target (y)
X = data.drop('MIC', axis=1)
y = data['MIC']

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Standarization:

In [ ]:
# Padronizando as características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Logistic Regression:

In [ ]:
# Inicializando e treinando o modelo
model = LogisticRegression()
model.fit(X_train, y_train)

# Fazendo previsões
y_pred = model.predict(X_test)

# Avaliando o modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')

# Supper Vector Regression:

In [ ]:
# Inicializando e treinando o modelo
model = SVR(kernel='rbf')  # 'rbf' é o kernel Radial Basis Function
model.fit(X_train, y_train)

# Fazendo previsões
y_pred = model.predict(X_test)

# Avaliando o modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R²: {r2}')


# Polinomial Regression:

In [ ]:
# Inicializando o transformador polinomial e o modelo
poly = PolynomialFeatures(degree=3)  # Grau do polinômio
model = LinearRegression()

# Criando o pipeline
pipeline = make_pipeline(poly, model)
pipeline.fit(X_train, y_train)

# Fazendo previsões
y_pred = pipeline.predict(X_test)

# Avaliando o modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')